import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise.dataset import BUILTIN_DATASETS #с помощью данного объекта мы можем использовать встроенные датасеты

data = Dataset.load_from_file(
    "u.data.txt",
    reader=Reader(line_format="user item rating timestamp", sep="\t"),
)

df = pd.DataFrame(data.raw_ratings, columns=['userId', 'movieId', 'rating', 'timestamp'])

df.nunique()

df['rating'].value_counts()

from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.25, random_state=13)
len(testset)

from surprise import SVD, KNNBasic, accuracy
sim_options = {
    'name': 'cosine',
    'user_based': False
}

knn = KNNBasic(sim_options=sim_options)
knn.fit(trainset)
predictions = knn.test(testset)
predictions

for prediction in predictions:
    if prediction.uid == '500' and prediction.iid == '699':
        print(prediction.r_ui)
        print(round(prediction.est, 2))
        break

accuracy.rmse(predictions)

uid = str(500)
iid = str(699)
pred = knn.predict(uid, iid, verbose=True)

pred = pd.DataFrame(predictions)
pred.sort_values(by=['est'],inplace=True,ascending = False)

recom = pred[pred.uid =='849']['iid'].to_list()

sim_options = {
    'name': 'cosine',
    'user_based': True
}

knn = KNNBasic(sim_options=sim_options)

knn.fit(trainset)
predictions = knn.test(testset)
accuracy.rmse(predictions)

model = SVD()
model.fit(trainset)
predictions = model.test(testset)
accuracy.rmse(predictions)

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k

ratings = pd.read_csv('ratings.csv') # Поставленные оценки
books = pd.read_csv('books.csv') # Информация о книгах
tags = pd.read_csv('tags.csv') # Информация о тегах
book_tags = pd.read_csv('book_tags.csv') # Книги с тегами

dict_map = dict(zip(books.goodreads_book_id,books.book_id))
book_tags['id'] = book_tags.goodreads_book_id.apply(lambda x: dict_map[x])

book_tags[book_tags['goodreads_book_id']==5]

book_tags = book_tags[book_tags.tag_id.isin(tags.tag_id)]
book_tags.shape

from scipy.sparse import csr_matrix

ratings_matrix = csr_matrix((ratings.rating,(ratings.user_id,ratings.book_id)))
# Передаём в качестве аргументов в функцию выставленный рейтинг (это будут значения матрицы),
# а также id пользователя и id книги (это будут индексы для строк и столбцов матрицы)
meta_matrix = csr_matrix(([1]*len(book_tags),(book_tags.id,book_tags.tag_id)))

ratings_matrix.mean()

model = LightFM(
    loss='warp-kos', # Определяем функцию потерь
    random_state=42, # Фиксируем случайное разбиение
    learning_rate=0.05, # Темп обучения
    no_components=100 # Размерность вектора для представления данных в модели
)

train, test = random_train_test_split(
    ratings_matrix, # Общая выборка
    test_percentage=0.2, # Размер тестовой выборки
    random_state=42 # Генератор случайных чисел
)

model = model.fit(
    train, # Обучающая выборка
    item_features=meta_matrix, # Признаки товаров
    epochs=10, # Количество эпох
    verbose=True,# Отображение обучения
    num_threads =6
)

prec_score = precision_at_k(
                     model,
                     test,
                     item_features = meta_matrix).mean()
print(prec_score)